Hi everyone,

Attached you have a dropbox link with the option data. A few things to note:
- You only have end of month date
- Option returns are delta hedged ONLY ONCE not on a daily basis. I went for this approach as it was easier to code so you will have the data faster.
- Only options which have prices at the next end of month are considered, again I did this because it was faster to code but it will probably not inhibit your results too much. 
- I have pre cleaned the option data for you. See for example Goyal (2009) for more details

For now the data seems super small (700mb) so I send you the whole data.

Best,
Patrick

In [21]:
# enable auto complete
%config Completer.use_jedi = False

In [14]:
import pandas as pd 

In [26]:
from datetime import datetime

In [15]:
pd.set_option('display.max_columns', 100)

# Data Exploration

In [16]:
sp500_op_ret = pd.read_csv("sp500_op_ret.csv")

In [17]:
mapping_table = pd.read_csv("mapping_table.csv")

In [18]:
#  PERMNO is a unique stock (share class) level identifier.
mapping_table.sample(10)

,secid,sdate,edate,permno
15267,123094,2005-02-14 00:00:00,2007-11-13 00:00:00,90566
12679,111508,2000-03-17 00:00:00,2001-03-22 00:00:00,87824
24510,209377,2016-04-27 00:00:00,2020-12-31 00:00:00,17627
25276,211138,2017-03-22 00:00:00,2020-12-31 00:00:00,16622
25285,211147,2017-03-27 00:00:00,2020-03-27 00:00:00,16618
13790,113443,2002-01-02 00:00:00,2020-12-31 00:00:00,89252
12760,111602,1996-01-01 00:00:00,2005-10-07 00:00:00,78856
22647,204385,2014-06-05 00:00:00,2020-12-31 00:00:00,14747
8478,106477,1996-01-01 00:00:00,2019-10-24 00:00:00,46042
24253,208564,2015-12-01 00:00:00,2020-12-31 00:00:00,15892


In [4]:
print(sp500_op_ret.columns)

Index(['secid', 'date', 'exdate', 'cp_flag', 'strike_price', 'best_bid',
       'best_offer', 'volume', 'open_interest', 'impl_volatility', 'delta',
       'gamma', 'vega', 'theta', 'optionid', 'cfadj', 'days_no_trading',
       'days_to_exp', 'forwardprice', 'spotprice', 'adj_spot', 'ir_rate',
       'mid_price', 'option_ret'],
      dtype='object')


In [7]:
sp500_op_ret.head()

,secid,date,exdate,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,optionid,cfadj,days_no_trading,days_to_exp,forwardprice,spotprice,adj_spot,ir_rate,mid_price,option_ret
0,5015,1996-01-31 00:00:00,1996-07-20 00:00:00,C,20.0,4.625,5.000,15,25,0.380062,0.793831,0.044693,4.456725,-2.603545,10980421,1,0,171,24.142672,24.000,24.0000,0.052253,4.8125,-0.025367
1,5048,1996-01-31 00:00:00,1996-08-17 00:00:00,P,40.0,3.000,3.375,5,37,0.245716,-0.507195,0.058756,11.186040,-1.508168,11613022,1,0,199,39.344930,39.000,39.0000,0.051780,3.1875,0.002347
2,5049,1996-01-31 00:00:00,1996-05-18 00:00:00,C,65.0,5.875,6.375,4,420,0.167298,0.833206,0.039200,9.020193,-5.515290,11618802,1,0,108,70.168661,70.375,35.1875,0.053274,6.1250,-0.009721
3,5049,1996-01-31 00:00:00,1996-08-17 00:00:00,C,70.0,3.250,3.625,13,3806,0.149214,0.559395,0.050652,19.823520,-4.653782,11770703,1,0,199,70.393348,70.375,35.1875,0.051780,3.4375,0.001186
4,5061,1996-01-31 00:00:00,1996-06-22 00:00:00,C,30.0,3.375,3.750,5,254,0.324523,0.657941,0.056807,7.146084,-3.929316,10914516,1,0,143,32.001209,31.750,31.7500,0.052726,3.5625,-0.006649


In [11]:
sum(sp500_op_ret["cp_flag"] == "C")

2179336

In [12]:
sum(sp500_op_ret["cp_flag"] == "P")

1646195

In [13]:
sp500_op_ret.describe()

,secid,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,optionid,cfadj,days_no_trading,days_to_exp,forwardprice,spotprice,adj_spot,ir_rate,mid_price,option_ret
count,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825531e+06,3.825393e+06
mean,1.118091e+05,1.583552e+02,1.048743e+01,1.099611e+01,8.288530e+01,1.758055e+03,3.903697e-01,1.159395e-01,3.800090e-02,2.467463e+01,-1.687396e+01,8.287611e+07,1.053916e+00,2.337976e-03,1.257383e+02,1.615546e+02,1.612870e+02,2.160093e+02,1.801193e-02,1.074177e+01,7.061732e-02
std,1.956878e+04,3.534783e+02,3.874286e+01,3.929525e+01,5.716760e+02,5.857573e+03,2.047089e-01,4.807320e-01,4.315247e-02,6.098045e+01,4.313121e+01,4.649504e+07,5.263510e-01,5.339109e-02,8.291730e+01,3.528938e+02,3.521467e+02,4.140947e+03,1.915548e-02,3.901709e+01,4.627202e+01
min,5.015000e+03,5.000000e-01,1.000000e-02,7.000000e-02,1.000000e+00,1.000000e+00,1.114700e-02,-9.996790e-01,0.000000e+00,4.900000e-04,-1.061745e+03,1.000001e+07,1.000000e+00,0.000000e+00,2.800000e+01,5.000000e-05,1.500000e-01,1.171875e-01,8.914000e-04,4.000000e-02,-1.601024e+04
25%,1.034340e+05,3.700000e+01,1.150000e+00,1.350000e+00,3.000000e+00,6.800000e+01,2.594700e-01,-2.718040e-01,1.047900e-02,5.147422e+00,-1.307261e+01,3.264821e+07,1.000000e+00,0.000000e+00,5.000000e+01,3.757191e+01,3.754000e+01,3.197000e+01,2.672931e-03,1.250000e+00,-3.809121e-02
50%,1.073980e+05,6.500000e+01,2.900000e+00,3.200000e+00,1.100000e+01,3.160000e+02,3.396460e-01,1.604910e-01,2.637400e-02,9.966705e+00,-6.169045e+00,1.033082e+08,1.000000e+00,0.000000e+00,1.090000e+02,6.498800e+01,6.486000e+01,5.673000e+01,1.074308e-02,3.050000e+00,-1.026760e-02
75%,1.116520e+05,1.290000e+02,7.400000e+00,7.900000e+00,4.000000e+01,1.264000e+03,4.580080e-01,5.068830e-01,5.078000e-02,2.019212e+01,-3.281715e+00,1.240800e+08,1.000000e+00,0.000000e+00,1.730000e+02,1.312548e+02,1.311500e+02,1.174850e+02,2.571517e-02,7.650000e+00,1.373601e-02
max,2.149050e+05,5.500000e+03,2.692950e+03,2.707550e+03,2.561690e+05,7.795400e+05,2.998749e+00,1.000030e+00,2.643942e+00,1.354885e+03,1.771834e+01,1.439230e+08,5.000000e+01,2.000000e+00,3.590000e+02,3.516114e+03,3.507070e+03,4.390544e+05,7.199510e-02,2.700250e+03,7.425289e+04


In [22]:
sp500_op_ret["impl_volatility"].mean()

0.3903697014422311

In [40]:
days2maturity = (    
    sp500_op_ret["exdate"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
  - sp500_op_ret["date"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
)

In [41]:
days2maturity.describe()

count                        3825531
mean     125 days 17:43:05.652344728
std       82 days 22:00:54.621799602
min                 28 days 00:00:00
25%                 50 days 00:00:00
50%                109 days 00:00:00
75%                173 days 00:00:00
max                359 days 00:00:00
dtype: object

In [43]:
days2maturity.median()

Timedelta('109 days 00:00:00')

In [38]:
datetime.strptime(sp500_op_ret["date"][0], '%Y-%m-%d %H:%M:%S') - datetime.strptime(sp500_op_ret["date"][100000], '%Y-%m-%d %H:%M:%S') 

datetime.timedelta(-790)

In [39]:
sp500_op_ret["date"][100000]

'1998-03-31 00:00:00'